- HttpConnectinPool:
    - 原因：
        - 1.短时间内发起了高频的请求导致ip被禁
        - 2.http连接池中的连接资源被耗尽
    - 解决：
        - 1.代理
        - 2.headers中加入Conection：“close”

- 代理：代理服务器，可以接受请求然后将其转发。
- 匿名度
    - 高匿：啥也不知道
    - 匿名：知道你使用了代理，但是不知道你的真实ip
    - 透明：知道你使用了代理并且知道你的真实ip
- 类型：
    - http
    - https
- 免费代理：
    - www.goubanjia.com
    - 快代理
    - 西祠代理
    - http://http.zhiliandaili.cn/
    
- cookie的处理

In [2]:
import requests
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}

In [4]:
url = 'https://www.baidu.com/s?wd=ip'
page_text = requests.get(url,headers=headers,proxies={'https':'111.231.94.44:8888'}).text
with open('ip.html','w',encoding='utf-8') as fp:
    fp.write(page_text)

In [ ]:
#代理池：列表
import random
proxy_list = [
    {'https':'121.231.94.44:8888'},
    {'https':'131.231.94.44:8888'},
    {'https':'141.231.94.44:8888'}
]
url = 'https://www.baidu.com/s?wd=ip'
page_text = requests.get(url,headers=headers,proxies=random.choice(proxy_list)).text
with open('ip.html','w',encoding='utf-8') as fp:
    fp.write(page_text)

In [5]:
from lxml import etree

In [28]:
#从代理精灵中提取代理ip
ip_url = 'http://t.11jsq.com/index.php/api/entry?method=proxyServer.generate_api_url&packid=1&fa=0&fetch_key=&groupid=0&qty=4&time=1&pro=&city=&port=1&format=html&ss=5&css=&dt=1&specialTxt=3&specialJson=&usertype=2'
page_text = requests.get(ip_url,headers=headers).text
tree = etree.HTML(page_text)
ip_list = tree.xpath('//body//text()')
print(ip_list)

['407 Proxy Authentication Required', 'IP:<123.112.23.183> 不在白名单内或者IP已经过了有效期失效不能访问，请添加白名单或者提取更长时效性的IP,在官网登录后在用户中心添加']


In [21]:
import random

In [22]:
#爬取西祠代理
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
    'Connection':"close"
}
url = 'https://www.xicidaili.com/nn/%d'
proxy_list_http = []
proxy_list_https = []
for page in range(1,20):
    new_url = format(url%page)
    ip_port = random.choice(ip_list)
    page_text = requests.get(new_url,headers=headers,proxies={'https':ip_port}).text
    tree = etree.HTML(page_text)
    #tbody不可以出现在xpath表达式中
    tr_list = tree.xpath('//*[@id="ip_list"]//tr')[1:]
    for tr in tr_list:
        ip = tr.xpath('./td[2]/text()')[0]
        port = tr.xpath('./td[3]/text()')[0]
        t_type = tr.xpath('./td[6]/text()')[0]
        ips = ip+':'+port
        if t_type == 'HTTP':
            dic = {
                t_type: ips
            }
            proxy_list_http.append(dic)
        else:
            dic = {
                t_type:ips
            }
            proxy_list_https.append(dic)
print(len(proxy_list_http),len(proxy_list_https))

ProxyError: HTTPSConnectionPool(host='www.xicidaili.com', port=443): Max retries exceeded with url: /nn/12 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000269A840DCC0>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',)))

In [ ]:
#检测
for ip in proxy_list_http:
    response = requests.get('https://www/sogou.com',headers=headers,proxies={'https':ip})
    if response.status_code == '200':
        print('检测到了可用ip')

- cookie的处理
    - 手动处理：将cookie封装到headers中
    - 自动处理：session对象。可以创建一个session对象，改对象可以像requests一样进行请求发送。不同之处在于如果在使用session进行请求发送的过程中产生了cookie，则cookie会被自动存储在session对象中。

In [32]:
#对雪球网中的新闻数据进行爬取https://xueqiu.com/
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
#     'Cookie':'aliyungf_tc=AQAAAAl2aA+kKgkAtxdwe3JmsY226Y+n; acw_tc=2760822915681668126047128e605abf3a5518432dc7f074b2c9cb26d0aa94; xq_a_token=75661393f1556aa7f900df4dc91059df49b83145; xq_r_token=29fe5e93ec0b24974bdd382ffb61d026d8350d7d; u=121568166816578; device_id=24700f9f1986800ab4fcc880530dd0ed'
}
url = 'https://xueqiu.com/v4/statuses/public_timeline_by_category.json?since_id=-1&max_id=20349203&count=15&category=-1'
page_text = requests.get(url=url,headers=headers).json()
page_text

{'error_code': '400016',
 'error_data': None,
 'error_description': '遇到错误，请刷新页面或者重新登录帐号后再试',
 'error_uri': '/v4/statuses/public_timeline_by_category.json'}

In [35]:
#创建session对象
session = requests.Session()
session.get('https://xueqiu.com',headers=headers)

url = 'https://xueqiu.com/v4/statuses/public_timeline_by_category.json?since_id=-1&max_id=20349203&count=15&category=-1'
page_text = session.get(url=url,headers=headers).json()
page_text

{'addition': [],
 'list': [{'category': 0,
   'column': '沪深',
   'data': '{"id":132614531,"title":"狼来了！今天，中囯电信行业打响第一枪！流量费用要降价了！","description":"狼，终究来了！ 刚刚传来大消息，中国工信部正式宣布：英国电信（BT）获得了在中国全国性经营通信的牌照。 随后，英国电信也在第一时间证实这一消息！他们兴高采烈地表示：取得牌照，意味着英国电信在中国迈出重要的一步！ 是的，你没有看错：英国电信！这是英国最大的电信公司，也是一家有着超过...","target":"/3583653389/132614531","reply_count":61,"retweet_count":7,"topic_title":"狼来了！今天，中囯电信行业打响第一枪！流量费用要降价了！","topic_desc":"狼，终究来了！ 刚刚传来大消息， 中国工信部正式宣布：英...","topic_symbol":null,"topic_pic":null,"topic_pic_hd":null,"pic_type":0,"first_pic":"https://xqimg.imedao.com/16d1a039553293dc3fb51093.jpg!186x186.jpg","pic_size":{"width":2,"height":1},"pic":"https://xqimg.imedao.com/16d1a039553293dc3fb51093.jpg!345x210.webp,https://xqimg.imedao.com/16d1a039557293dd3fc67b4c.jpg!345x210.webp","pic_sizes":[{"width":1080,"height":675},{"width":854,"height":522}],"cover_pic":null,"user":{"id":3583653389,"profile":"/3583653389","description":"","following":false,"screen_name":"自由如风B","profile_image_url":"community/201

- 验证码的识别
    - 超级鹰：http://www.chaojiying.com/about.html
        - 注册：（用户中心身份）
        - 登陆：
            - 创建一个软件：899370
            - 下载示例代码
    - 打码兔
    - 云打码

In [39]:
import requests
from hashlib import md5

class Chaojiying_Client(object):

    def __init__(self, username, password, soft_id):
        self.username = username
        password =  password.encode('utf8')
        self.password = md5(password).hexdigest()
        self.soft_id = soft_id
        self.base_params = {
            'user': self.username,
            'pass2': self.password,
            'softid': self.soft_id,
        }
        self.headers = {
            'Connection': 'Keep-Alive',
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)',
        }

    def PostPic(self, im, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
        }
        params.update(self.base_params)
        files = {'userfile': ('ccc.jpg', im)}
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, files=files, headers=self.headers)
        return r.json()

    def ReportError(self, im_id):
        """
        im_id:报错题目的图片ID
        """
        params = {
            'id': im_id,
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/ReportError.php', data=params, headers=self.headers)
        return r.json()

In [41]:
#识别古诗文网中的验证码
def tranformImgData(imgPath,t_type):
    chaojiying = Chaojiying_Client('bobo328410948', 'bobo328410948', '899370')
    im = open(imgPath, 'rb').read()
    return chaojiying.PostPic(im, t_type)['pic_str']

url = 'https://so.gushiwen.org/user/login.aspx?from=http://so.gushiwen.org/user/collect.aspx'
page_text = requests.get(url,headers=headers).text
tree = etree.HTML(page_text)
img_src = 'https://so.gushiwen.org'+tree.xpath('//*[@id="imgCode"]/@src')[0]
img_data = requests.get(img_src,headers=headers).content
with open('./code.jpg','wb') as fp:
    fp.write(img_data)
    
tranformImgData('./code.jpg',1004)

'qh39'

In [45]:
#模拟登陆
s = requests.Session()
url = 'https://so.gushiwen.org/user/login.aspx?from=http://so.gushiwen.org/user/collect.aspx'
page_text = s.get(url,headers=headers).text
tree = etree.HTML(page_text)
img_src = 'https://so.gushiwen.org'+tree.xpath('//*[@id="imgCode"]/@src')[0]
img_data = s.get(img_src,headers=headers).content
with open('./code.jpg','wb') as fp:
    fp.write(img_data)
    
#动态获取变化的请求参数
__VIEWSTATE = tree.xpath('//*[@id="__VIEWSTATE"]/@value')[0]
__VIEWSTATEGENERATOR = tree.xpath('//*[@id="__VIEWSTATEGENERATOR"]/@value')[0]
    
code_text = tranformImgData('./code.jpg',1004)
print(code_text)
login_url = 'https://so.gushiwen.org/user/login.aspx?from=http%3a%2f%2fso.gushiwen.org%2fuser%2fcollect.aspx'
data = {
    '__VIEWSTATE': __VIEWSTATE,
    '__VIEWSTATEGENERATOR': __VIEWSTATEGENERATOR,
    'from':'http://so.gushiwen.org/user/collect.aspx',
    'email': 'www.zhangbowudi@qq.com',
    'pwd': 'bobo328410948',
    'code': code_text,
    'denglu': '登录',
}
page_text = s.post(url=login_url,headers=headers,data=data).text
with open('login.html','w',encoding='utf-8') as fp:
    fp.write(page_text)

hhsl


- 动态变化的请求参数
    - 通常情况下动态变化的请求参数都会被隐藏在前台页面源码中

In [6]:
from time import sleep
import time

In [7]:
from multiprocessing.dummy import Pool
start = time.time()
urls = [
    'http://127.0.0.1:5000/bobo',
    'http://127.0.0.1:5000/jay'
]
def get_request(url):
    page_text = requests.get(url).text
    print(page_text)
pool = Pool(3)
pool.map(get_request,urls)

print('总耗时：',time.time()-start)

Hello jayHello bobo

总耗时： 2.0196006298065186


In [5]:
from flask import Flask
from time import sleep
app = Flask(__name__)
@app.route('/index')
def index():
    sleep(2)
    return 'hello'
@app.route('/index1')
def index1():
    sleep(2)
    return 'hello1'
if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Sep/2019 11:35:03] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [11/Sep/2019 11:35:05] "GET /index1 HTTP/1.1" 200 -
127.0.0.1 - - [11/Sep/2019 12:07:02] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [11/Sep/2019 12:07:02] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [11/Sep/2019 12:11:18] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [11/Sep/2019 12:11:20] "GET /index1 HTTP/1.1" 200 -
127.0.0.1 - - [11/Sep/2019 12:21:46] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [11/Sep/2019 12:21:48] "GET /index1 HTTP/1.1" 200 -


In [3]:
import requests

In [ ]:
from multiprocessing.dummy import Pool
start = time.time()
urls = [
    'http://localhost:5000/index',
     'http://localhost:5000/index',
     'http://localhost:5000/index',
    
]
def get_request(url):
    
pool = Pool(3)
pool.map(get_request,urls)

print('总耗时：',time.time()-start)

### 单线程+多任务异步协程
- 协程
    - 在函数（特殊的函数）定义的时候，如果使用了async修饰的话，则改函数调用后会返回一个协程对象，并且函数内部的实现语句不会被立即执行
- 任务对象
    - 任务对象就是对协程对象的进一步封装。任务对象==高级的协程对象==特殊的函数
    - 任务对象时必须要注册到事件循环对象中
    - 给任务对象绑定回调：爬虫的数据解析中
- 事件循环
    - 当做是一个容器，容器中必须存放任务对象。
    - 当启动事件循环对象后，则事件循环对象会对其内部存储任务对象进行异步的执行。
- aiohttp:支持异步网络请求的模块